In [4]:
from schema_parser import SchemaParser
from schema_diff import SchemaDiff, json
from base import DATA_PATH

# schema_v1_parser = SchemaParser(f'{DATA_PATH}/schema-1-example.txt')
# schema_v1 = schema_v1_parser.parse()

# schema_v2_parser = SchemaParser(f'{DATA_PATH}/schema-2-example.txt')
# schema_v2 = schema_v2_parser.parse()

# schema_diff = SchemaDiff(schema_v1, schema_v2)
# change_report = schema_diff.detect_changes()
# print(json.dumps(change_report, indent=2))


In [51]:
import os
import json
from schema_parser import SchemaParser
from schema_diff import SchemaDiff

# Get list of all schema files
v1_schemas = sorted([f for f in os.listdir(DATA_PATH) if f.startswith('schema-1')])
v2_schemas = sorted([f for f in os.listdir(DATA_PATH) if f.startswith('schema-2')])

# Pair each schema-1 file with the corresponding schema-2 file
for v1_file, v2_file in zip(v1_schemas, v2_schemas):
    # Load schemas from .txt files
    schema_v1_parser = SchemaParser(os.path.join(DATA_PATH, v1_file))
    schema_v1 = schema_v1_parser.parse()
    print("schemav1")
    print(json.dumps(schema_v1, indent=2))

    schema_v2_parser = SchemaParser(os.path.join(DATA_PATH, v2_file))
    schema_v2 = schema_v2_parser.parse()
    print("schemav2")
    print(json.dumps(schema_v2, indent=2))

    # Generate schema diff
    schema_diff = SchemaDiff(schema_v1, schema_v2)
    change_report = schema_diff.detect_changes()

    # Print the JSON output for the detected changes
    # print(f"Change report for {v1_file} vs {v2_file}:")
    # print(json.dumps(change_report, indent=2))
    # print("\n" + "="*50 + "\n")


schemav1
{
  "Query": {
    "getProduct": {
      "parameters": {
        "id": "ID!"
      },
      "return_type": "Product"
    }
  },
  "Product": {
    "id": "ID",
    "name": "String",
    "price": "Float",
    "dimensions": "Dimensions",
    "manufacturer": "Manufacturer"
  },
  "Dimensions": {
    "length": "Float",
    "width": "Float",
    "height": "Float"
  },
  "Manufacturer": {
    "id": "ID",
    "name": "String",
    "location": "String"
  }
}
schemav2
{
  "Query": {
    "getProduct": {
      "parameters": {
        "id": "ID!"
      },
      "return_type": "Product"
    },
    "getManufacturer": {
      "parameters": {
        "id": "ID!"
      },
      "return_type": "Manufacturer"
    }
  },
  "Product": {
    "id": "ID",
    "name": "String",
    "price": "Float",
    "dimensions": "Dimensions",
    "manufacturer": "Manufacturer",
    "rating": "Int"
  },
  "Dimensions": {
    "length": "Float",
    "width": "Float",
    "height": "Float"
  },
  "Manufacturer": {
   

In [7]:
examples = ["example","example-1"]
example = examples[0]

schema_v1_parser = SchemaParser(f'{DATA_PATH}/schema-1-{example}.txt')
schema_v1 = schema_v1_parser.parse()

schema_v2_parser = SchemaParser(f'{DATA_PATH}/schema-2-{example}.txt')
schema_v2 = schema_v2_parser.parse()

output_path = (f'{DATA_PATH}/output-{example}.txt')
with open(output_path, 'r') as file:
    output_v1 = json.load(file)

In [8]:
import os
from dotenv import load_dotenv
import anthropic
from base import DATA_PATH

load_dotenv()  # Load environment variables from .env file

def create_message(prompt, api_key=os.environ.get("CLAUDE_API")):
    client = anthropic.Anthropic(api_key=api_key)

    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=2048,  # Set max_tokens to your desired value
        messages=[{"role": "user", "content": prompt}]
    )
    return message.content

example_one = """Input:
Schema v1: 
type Query {
  getUser(id: ID!): User
}

type User {
  id: ID
  name: String
  age: Int
  email: String
  phoneNumber: String
}

Schema v2:
type Query {
  getUser(id: ID!): User
  getUserByEmail(email: String!): User
}

type User {
  id: ID!
  name: String!
  email: String!
  phoneNumber: String
  isActive: Boolean
}

Output:
{
  "changes": [
    {
      "type": "User",
      "field": "id",
      "change": "Field 'id' made non-nullable",
      "breaking": true,
      "release_note": "The `id` field on `User` has been changed to non-nullable. This is a breaking change; any existing data where `id` is null will cause errors."
    },
    {
      "type": "User",
      "field": "name",
      "change": "Field 'name' made non-nullable",
      "breaking": true,
      "release_note": "The `name` field on `User` is now non-nullable. This is a breaking change, and clients should ensure all `User` entries include a `name` value."
    },
    {
      "type": "User",
      "field": "email",
      "change": "Field 'email' made non-nullable",
      "breaking": true,
      "release_note": "The `email` field on `User` has been updated to non-nullable, marking a breaking change. Ensure all `User` entries include an `email` value."
    },
    {
      "type": "User",
      "field": "isActive",
      "change": "Added new Boolean field 'isActive'",
      "breaking": false,
      "release_note": "A new `isActive` field has been added to the `User` type, allowing clients to check if a user is active. This addition is non-breaking."
    },
    {
      "type": "Query",
      "field": "getUserByEmail",
      "change": "Added new query 'getUserByEmail'",
      "breaking": false,
      "release_note": "A new query `getUserByEmail` has been added. Clients can now retrieve a user by email address. This change is non-breaking."
    }
  ],
  "release_notes": {
    "summary": "This release introduces several breaking changes, including updates to `User` fields (making `id`, `name`, and `email` non-nullable) and new additions such as the `isActive` field and `getUserByEmail` query."
  }
}
"""

example_two = """Input:
Schema v1:
type Query {
  listProducts(category: String): [Product]
}

type Product {
  id: ID!
  name: String!
  price: Float!
  inStock: Boolean
}

Schema v2:
type Query {
  listProducts(category: String, sortBy: String): [Product]
  getProduct(id: ID!): Product
}

type Product {
  id: ID!
  name: String!
  price: Float!
  currency: String
  stockCount: Int
}

Output:
{
  "changes": [
    {
      "type": "Query",
      "field": "listProducts",
      "change": "Added new optional input parameter 'sortBy'",
      "breaking": false,
      "release_note": "The `listProducts` query now accepts an optional `sortBy` parameter, allowing clients to sort products. This is a non-breaking change."
    },
    {
      "type": "Query",
      "field": "getProduct",
      "change": "Added new query 'getProduct'",
      "breaking": false,
      "release_note": "A new query `getProduct` has been introduced, enabling clients to fetch a product by its ID. This addition is non-breaking."
    },
    {
      "type": "Product",
      "field": "currency",
      "change": "Added new String field 'currency'",
      "breaking": false,
      "release_note": "The `currency` field has been added to the `Product` type, allowing clients to see the currency of the product's price. This is a non-breaking change."
    },
    {
      "type": "Product",
      "field": "stockCount",
      "change": "Added new Int field 'stockCount'",
      "breaking": false,
      "release_note": "A new field `stockCount` has been added to the `Product` type. Clients can now view the number of items in stock. This is a non-breaking change."
    }
  ],
  "release_notes": {
    "summary": "This release includes non-breaking enhancements to the `listProducts` query with a new `sortBy` parameter, new fields `currency` and `stockCount` in `Product`, and a new query `getProduct` for fetching product details by ID."
  }
}
"""

prompt = f"""You are a tool designed to detect differences between two GraphQL schemas. Generate a summary of breaking and non-breaking changes.

Given two GraphQL schemas (Schema v1 and Schema v2) as inputs, follow these steps:
1. Parse the schemas to identify differences such as added, removed, or changed fields, types, or parameters.
2. For each change, determine if it is a breaking or non-breaking change based on compatibility with existing queries.
3. Generate a natural language summary suitable for release notes based on these changes. Each entry should describe the change, the type and field affected, and specify if it is breaking or non-breaking.
4. Structure your response with an overall summary in the release notes and detail each change individually.

Return your response in JSON format with the following structure:

{{
  "changes": [
    {{
      "type": "<Type>",
      "field": "<Field>",
      "change": "<Description of the change>",
      "breaking": <true or false>,
      "release_note": "<Detailed explanation for release notes>"
    }},
    ...
  ],
  "release_notes": {{
    "summary": "<High-level summary of the release including both breaking and non-breaking changes>"
  }}
}}

Here are two example scenarios to guide your response:
{example_one}
{example_two}

Below are the GraphQL schemas to review:
Schema v1:
{schema_v1}

Schema v2:
{schema_v2}
"""

response = create_message(prompt)
print(response)


[TextBlock(text='{\n  "changes": [\n    {\n      "type": "Query",\n      "field": "listProducts",\n      "change": "Added new optional parameter \'inStock\' and made return type non-nullable",\n      "breaking": true,\n      "release_note": "The `listProducts` query now accepts an optional `inStock` parameter and its return type is now non-nullable ([Product]!). This is a breaking change as it affects the response type."\n    },\n    {\n      "type": "Query",\n      "field": "getManufacturer",\n      "change": "Added new query \'getManufacturer\'",\n      "breaking": false,\n      "release_note": "A new query `getManufacturer` has been added to fetch manufacturer details by ID. This is a non-breaking change."\n    },\n    {\n      "type": "Product",\n      "field": "id",\n      "change": "Field \'id\' made non-nullable",\n      "breaking": true,\n      "release_note": "The `id` field on `Product` is now non-nullable. This is a breaking change for clients expecting nullable IDs."\n    }

In [9]:
import json

def safe_load_json(response_text, default_value=None):
    """
    Safely loads JSON data from a response text.

    Args:
        response_text (str): The JSON string to be loaded.
        default_value (any): Value to return in case of an error (default: None).

    Returns:
        dict: The loaded JSON data if successful, otherwise default_value.
    """
    try:
        # Attempt to load the JSON data
        json_data = json.loads(response_text)
        return json_data
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")  # Log the error
        return default_value
    except Exception as e:
        print(f"An error occurred while loading JSON: {e}")  # Log unexpected errors
        return default_value

# Example usage
response_text = response[0].text  # Assuming response[0].text contains your JSON string
json_response = safe_load_json(response_text)


In [14]:
expected_output

{'changes': [{'type': 'Product',
   'field': 'id',
   'change': "Field 'id' made non-nullable",
   'breaking': True,
   'release_note': 'The `id` field on `Product` has been made non-nullable. This is a breaking change; existing entries with null `id` values will cause errors.'},
  {'type': 'Product',
   'field': 'name',
   'change': "Field 'name' made non-nullable",
   'breaking': True,
   'release_note': 'The `name` field on `Product` is now non-nullable. Ensure all product entries include a name value, as this is a breaking change.'},
  {'type': 'Product',
   'field': 'tags',
   'change': "List 'tags' made non-nullable",
   'breaking': True,
   'release_note': 'The `tags` list on `Product` now requires non-null values for each tag. Existing entries with null values in `tags` will cause errors, marking a breaking change.'},
  {'type': 'Product',
   'field': 'manufacturer',
   'change': "Field 'manufacturer' made non-nullable",
   'breaking': True,
   'release_note': 'The `manufacture

In [13]:
json_response

{'changes': [{'type': 'Query',
   'field': 'listProducts',
   'change': "Added new optional parameter 'inStock' and made return type non-nullable",
   'breaking': True,
   'release_note': 'The `listProducts` query now accepts an optional `inStock` parameter and its return type is now non-nullable ([Product]!). This is a breaking change as it affects the response type.'},
  {'type': 'Query',
   'field': 'getManufacturer',
   'change': "Added new query 'getManufacturer'",
   'breaking': False,
   'release_note': 'A new query `getManufacturer` has been added to fetch manufacturer details by ID. This is a non-breaking change.'},
  {'type': 'Product',
   'field': 'id',
   'change': "Field 'id' made non-nullable",
   'breaking': True,
   'release_note': 'The `id` field on `Product` is now non-nullable. This is a breaking change for clients expecting nullable IDs.'},
  {'type': 'Product',
   'field': 'name',
   'change': "Field 'name' made non-nullable",
   'breaking': True,
   'release_note'

In [10]:
import json
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Example expected output for evaluation
expected_output = output_v1

# Function to evaluate BLEU score
def evaluate_bleu(expected, generated):
    return sentence_bleu([expected], generated)

# Function to evaluate ROUGE score
def evaluate_rouge(expected, generated):
    rouge = Rouge()
    scores = rouge.get_scores(generated, expected, avg=True)
    return scores

# Evaluate Accuracy (for exact match)
def evaluate_exact_match(expected, generated):
    return expected == generated

# Perform evaluations
def evaluate_response(expected, generated):
    # Extract summaries for BLEU and ROUGE evaluation
    expected_summary = expected["release_notes"]["summary"]
    generated_summary = generated["release_notes"]["summary"]
    
    # Evaluate metrics
    bleu_score = evaluate_bleu(expected_summary.split(), generated_summary.split())
    rouge_score = evaluate_rouge(expected_summary, generated_summary)
    
    exact_match = evaluate_exact_match(json.dumps(expected), json.dumps(generated))
    
    # Print results
    print(f"BLEU Score: {bleu_score:.4f}")
    print(f"ROUGE Score: {rouge_score}")
    print(f"Exact Match: {exact_match}")

# Evaluate the model response
evaluate_response(expected_output, json_response)


BLEU Score: 0.0000
ROUGE Score: {'rouge-1': {'r': 0.5161290322580645, 'p': 0.25806451612903225, 'f': 0.3440860170609319}, 'rouge-2': {'r': 0.08333333333333333, 'p': 0.0379746835443038, 'f': 0.05217390874253344}, 'rouge-l': {'r': 0.41935483870967744, 'p': 0.20967741935483872, 'f': 0.27956988802867394}}
Exact Match: False


/opt/homebrew/Caskroom/mambaforge/base/envs/graphql-schema-diff-service/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/homebrew/Caskroom/mambaforge/base/envs/graphql-schema-diff-service/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
